In [42]:
import pickle
import math
import sys
sys.path.append('..')
from src.sympy_utilities import *
from src.BinaryEncodings import *
from src.qiskit_utilities import *

from qiskit import QuantumCircuit, transpile

from scipy.linalg import expm
import matplotlib.pyplot as plt

h_b_HO = 0.5*(p*p + m*m*q*q)
h_f_HO = 0.5*(m*b*bdag - m*bdag*b)
cutoff=2**3
hamHO = Hamiltonian(h_b_HO, h_f_HO, {m:1, g:1},
                  cutoff, standard_encode)

In [43]:
eigSys = np.linalg.eig(hamHO.hamMatrix)
eVals = eigSys[0]
minIdx = np.where(eVals == np.min(eVals))[0][0] # just take the first ground state
print(minIdx)

1


In [44]:
print(eigSys[1][minIdx])

[0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [45]:
state=eigSys[1][minIdx]

for t in range(0,10):
    print(np.vdot(state,np.matmul(expm(-1j*hamHO.hamMatrix*t),state)))


(1+0j)
(1+4.440892098500626e-16j)
(1+8.881784197001252e-16j)
(1+1.3322676295501878e-15j)
(1+1.7763568394002505e-15j)
(1+2.220446049250313e-15j)
(1+2.6645352591003757e-15j)
(1+3.1086244689504383e-15j)
(1+3.552713678800501e-15j)
(1+3.9968028886505635e-15j)


In [46]:
for state in eigSys[1]:
    print( np.vdot(state,np.matmul(expm(-1j*hamHO.hamMatrix*1),state)) )

(-0.4161468365471422-0.9092974268256818j)
(1+4.440892098500626e-16j)
(0.960170286650366+0.27941549819892586j)
(-0.6536436208636119+0.7568024953079282j)
(-0.6536436208636119+0.7568024953079282j)
(-0.4161468365471424-0.9092974268256817j)
(-0.14550003380861354-0.9893582466233818j)
(0.960170286650366+0.27941549819892586j)
(-0.9899924966004454-0.1411200080598672j)
(0.5403023058681399-0.8414709848078964j)
(0.7539022543433046-0.6569865987187891j)
(0.28366218546322625+0.9589242746631385j)
(0.28366218546322625+0.9589242746631385j)
(-0.9899924966004454-0.1411200080598672j)
(-0.9111302618846769-0.4121184852417566j)
(0.7539022543433046-0.6569865987187891j)


In [47]:
trotterStep = pauli_string_to_trotter_step(hamHO.pauliStrings,1)
nq=int(math.ceil(math.log(cutoff,2)))+1
qc = QuantumCircuit(nq,nq)
qc.append(trotterStep, [i for i in range(nq)])

In [48]:
from qiskit import (execute, Aer)
from qiskit import ClassicalRegister
from qiskit.visualization import plot_histogram

for t in range(0,10):
    trotterStep = pauli_string_to_trotter_step(hamHO.pauliStrings,t)
    nq=int(math.ceil(math.log(cutoff,2)))+1
    qc = QuantumCircuit(nq,nq)
    qc.append(trotterStep, [i for i in range(nq)])
    simulator = Aer.get_backend('qasm_simulator')
    cr = ClassicalRegister(nq,'nc')
    qc.add_register(cr)
    qc.measure([i for i in range(nq)],[i for i in range(nq)])
    job = execute(qc, simulator, shots=1000)
    result = job.result()
    counts = result.get_counts(qc)
    print(counts)

{'0000 0000': 1000}
{'0000 0000': 1000}
{'0000 0000': 1000}
{'0000 0000': 1000}
{'0000 0000': 1000}
{'0000 0000': 1000}
{'0000 0000': 1000}
{'0000 0000': 1000}
{'0000 0000': 1000}
{'0000 0000': 1000}
